In [1]:
#These are the libraries you can use.  You may add any libraries directy related to threading if this is a direction
#you wish to go (this is not from the course, so it's entirely on you if you wish to use threading).  Any
#further libraries you wish to use you must email me, james@uwaterloo.ca, for permission.

from IPython.display import display, Math, Latex

import pandas as pd
import numpy as np
import numpy_financial as npf
import yfinance as yf
import matplotlib.pyplot as plt
import random
from datetime import datetime
import pandas as pd
import numpy as np


In [2]:
# download these libraries if you don't have them
#%pip install PyPortfolioOpt

%pip install PyPortfolioOpt


Note: you may need to restart the kernel to use updated packages.


In [3]:
from pypfopt import EfficientFrontier
from pypfopt import risk_models
from pypfopt import expected_returns
import cvxpy as cp
from pypfopt import BlackLittermanModel



## Group Assignment
### Team Number: 14
### Team Member Names: Aadya, Furqan, Pranay
### Team Strategy Chosen: Market Beat

In [4]:

input_tickers = pd.read_csv("Tickers_Example.csv", names=['Tickers']) #OR any other CSV file with tickers in it
input_tickers

,Tickers
0,AAPL
1,AAPL
2,ABBV
3,ABT
4,ABT
5,ACN
6,HSBC
7,AGN
8,AIG
9,AMZN


In [5]:
## PRELIMINARY DATA PROCESSING 
## AND INPUT PROCESSING

## WILL CSV BE EXACTLY THE WAY IT IS IN EXAMPLE??

## Check if implementation of US and Canadian ticker validation is sufficient/correct
## Should we do it through currency checks, or exchange checks as below?

def validate_ticker(ticker):
    try:
        stock = yf.Ticker(ticker)
        info = stock.info

        if info and 'regularMarketPrice' in info:
            if 'exchange' in info:
                exchange = info['exchange']
                # yfinance uses these exchange codes, so we check against them
                valid_exchanges = ['NMS', 'NGM', 'NYQ','NAD','PCX','BTS','CBQ','ASE', 'TOR', 'TSE', 'Toronto', 'TSXV', 'CNQ', 'CSE','NEO']
                if exchange in valid_exchanges:
                    return True
                else:
                    return False  
            else:
                return False  
        else:
            return False
    except:
        return False

valid_tickers = []
for ticker in input_tickers['Tickers']:
    if validate_ticker(ticker):
        valid_tickers.append(ticker)
        #print(f"✓ Valid: {ticker}")  
    else:
        print(f"✗ Invalid ticker: {ticker}")

valid_tickers_df = pd.DataFrame(valid_tickers, columns=['Ticker'])
valid_tickers_df

# ✗ Invalid ticker: AGN
# ✗ Invalid ticker: MC.PA
# ✗ Invalid ticker: CELG
# ✗ Invalid ticker: 9984.T
# ✗ Invalid ticker: MON
# ✗ Invalid ticker: RELIANCE.NS
# ✗ Invalid ticker: RTN
# ✗ Invalid ticker: INFY.NS
# ✗ Invalid ticker: CPA.AX

✗ Invalid ticker: AGN
✗ Invalid ticker: MC.PA
✗ Invalid ticker: CELG
✗ Invalid ticker: 9984.T
✗ Invalid ticker: MON
✗ Invalid ticker: RELIANCE.NS
✗ Invalid ticker: RTN
✗ Invalid ticker: INFY.NS
✗ Invalid ticker: CPA.AX


,Ticker
0,AAPL
1,AAPL
2,ABBV
3,ABT
4,ABT
5,ACN
6,HSBC
7,AIG
8,AMZN
9,AXP


In [6]:
# check for duplicate tickers

def check_duplicates(df):
    duplicates = df.duplicated()

    for index in duplicates.index:
        if duplicates[index] == True:
            print(f"Removing duplicate: {df.loc[index, 'Ticker']}")
            # remove the row from the dataframe
            df.drop(index, inplace=True)
    return df.reset_index().drop(columns=['index'])

check_duplicates(valid_tickers_df)

Removing duplicate: AAPL
Removing duplicate: ABT
Removing duplicate: BA


,Ticker
0,AAPL
1,ABBV
2,ABT
3,ACN
4,HSBC
5,AIG
6,AMZN
7,AXP
8,BA
9,BAC


In [7]:
## REMOVING STOCKS WITH AVG TRADE VOLUME < 5000 between Oct 1, 2024 and Sep 30, 2025 (drop months with < 18 trading days).
##CHECK to ensure, and filter out stocks that do not meet this criteria

def filter_by_avg_volume(df, start_date="2024-10-01", end_date="2025-09-30", min_avg_volume=5000, min_trading_days=18):
    filtered_tickers = []
    for ticker in df['Ticker']:
        stock = yf.Ticker(ticker)
        
        hist = stock.history(start=start_date, end=end_date)
        hist.index = hist.index.tz_localize(None)

        monthly_groups = hist.groupby(hist.index.to_period("M"))
        valid = True
        for month, group in monthly_groups:
            trading_days = len(group)
            if trading_days >= min_trading_days:
                avg_volume = group['Volume'].mean()
                if avg_volume < min_avg_volume:
                    print(f"✗ Ticker failed volume filter: {ticker} (Avg Volume: {avg_volume:.2f} in {month})")
                    valid = False
                    break
        if valid:
            print(f"✓ Ticker passed volume filter: {ticker}")
            filtered_tickers.append(ticker)
    return pd.DataFrame(filtered_tickers, columns=['Ticker'])

final_tickers_df = filter_by_avg_volume(valid_tickers_df)
final_tickers_df


✓ Ticker passed volume filter: AAPL
✓ Ticker passed volume filter: ABBV
✓ Ticker passed volume filter: ABT
✓ Ticker passed volume filter: ACN
✓ Ticker passed volume filter: HSBC
✓ Ticker passed volume filter: AIG
✓ Ticker passed volume filter: AMZN
✓ Ticker passed volume filter: AXP
✓ Ticker passed volume filter: BA
✓ Ticker passed volume filter: BAC
✓ Ticker passed volume filter: BB.TO
✓ Ticker passed volume filter: BIIB
✓ Ticker passed volume filter: BK
✓ Ticker passed volume filter: SAP
✓ Ticker passed volume filter: BLK
✓ Ticker passed volume filter: BMY
✓ Ticker passed volume filter: C
✓ Ticker passed volume filter: CAT
✓ Ticker passed volume filter: CL
✓ Ticker passed volume filter: SAN
✓ Ticker passed volume filter: KO
✓ Ticker passed volume filter: LLY
✓ Ticker passed volume filter: LMT
✓ Ticker passed volume filter: MO
✓ Ticker passed volume filter: MRK
✓ Ticker passed volume filter: PEP
✓ Ticker passed volume filter: PFE
✓ Ticker passed volume filter: PG
✓ Ticker passed volum

,Ticker
0,AAPL
1,ABBV
2,ABT
3,ACN
4,HSBC
5,AIG
6,AMZN
7,AXP
8,BA
9,BAC


In [8]:
#Downloading history for valid tickers

tickers_list =  valid_tickers_df['Ticker'].tolist()

stock_data= yf.download(tickers_list, period="5y", interval="1d", group_by='ticker', auto_adjust=True, threads=True)

# Download benchmark data (S&P 500 + TSX average)
sp500_data = yf.download('^GSPC', period='5y', auto_adjust=True)
tsx_data = yf.download('^GSPTSE', period='5y', auto_adjust=True)

# checking the 'Close' prices to ensure they are Series
if isinstance(sp500_data, pd.DataFrame):
    sp500 = sp500_data['Close'].squeeze() 
else:
    sp500 = sp500_data

if isinstance(tsx_data, pd.DataFrame):
    tsx = tsx_data['Close'].squeeze()  
else:
    tsx = tsx_data

# Remove timezones
sp500.index = sp500.index.tz_localize(None)
tsx.index = tsx.index.tz_localize(None)




[*********************100%***********************]  41 of 41 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


Scenario 1: High Alpha, Low Beta (< 1.0)

 - Stock outperforms with less volatility than market

 - Best case/ideal scenario 

 - Means it is a Defensive stock that somehow beats the market, low risk high return

Scenario 2: High Alpha, High Beta (> 1.5)

 - Stock outperforms but is very volatile

 - Risky but rewarding

 - Also high reward, but higher risk as well. Still viable option to help choose stocks that will beat market

 - Example: Tech stock in a bull market

Scenario 3: Low/Negative Alpha, High Beta

 - Stock underperforms and is volatile

 - Low return relative to market/less than market, AND high volatility so potential further loss on an already underperforming stock.  

 - Worst case 

 - Want to avoid these types of stocks 


WE WANT TO INVEST IN:

Stocks which fall within the first two scenarios, as they are the most ideal to maximize returns, scenario 1 being the most effective. Stocks which fall in scenario 3, will be avoided. 

In [9]:
## CALCULATING ALPHA AND SORTING TICKERS HIGH TO LOW BY ALPHA


#assuming we have a list of valid tickers
#we need a function that simply returns the alpha of a ticker
#then run a for loop that calls that function for every ticker
#while ranking it from best to worst in a list or dictionary maybe
#going to also assume we have the portfolio returns, risk free rate, expected market return, and the portfolio beta


# calculates beta of a stock 
# beta is cov (Rp, Rm) / var (Rm)
def calc_beta(ticker_returns, market_returns):
    covariance = ticker_returns.cov(market_returns)
    variance = market_returns.var()
    beta = covariance / variance
    return beta

#function to calculate alpha of a ticker given the following parameters
#returns the ticker and its alpha in a list
#Good alpha > 0; Means positive returns relative to the market
#Zero alpha (\(\alpha =0\)): The investment's return was in line with the benchmark index, meaning it did not outperform or underperform after adjusting for risk.
#Negative alpha (\(<0\)): The investment's performance was worse than its benchmark index, considering the risk taken. 
def calc_alpha(portfolio_return, risk_free_rate, expected_market_return, portfolio_beta): 
    tick_alpha_calculation = portfolio_return - (risk_free_rate + (portfolio_beta * (expected_market_return-risk_free_rate)))

    return tick_alpha_calculation

# calculating sharpe ratio 
# sharpe ratio of a stock is = (Rp - Rf) / σp
# Good sharpe ratio > 1: indicates good risk-adjusted performance
# Average sharpe ratio ~ 0.5 - 1: indicates moderate risk-adjusted performance
# Poor sharpe ratio < 0.5: indicates poor risk-adjusted performance
# if ticker_returns are daily, then risk free has to be daily as well
def calc_sharpe(ticker_returns, risk_free_rate):
    std = ticker_returns.std()
    mean_return = ticker_returns.mean()
    sharpe_ratio = (mean_return - risk_free_rate) / std
    return sharpe_ratio

# calculating sortino ratio 
# sortino ratio = (Rp - Rf) / σd when σd is the standard deviation of negative asset returns
# sortino ratio focuses only on downside volatility which helps eliminate 
# the risk of downside movements in asset prices
# Intuitively good return per unit of bad volatility
# Good sortino ratio > 0: good risk-adjusted performance with low downside 
# Zero sortino ratio = 0: no excess return relative to downside risk
# Low sortino ratio < 0: poor risk-adjusted performance with high downside risk

def calc_sortino(ticker_returns, risk_free_rate):
    downside_dev = ticker_returns[ticker_returns < 0].std()
    mean_return = ticker_returns.mean()
    sortino_ratio = (mean_return - risk_free_rate) / downside_dev
    return sortino_ratio


##Calculating momentum, shows how much a stock has returned relative to a chosen date/period a while back
#Determining the change in return a given stock has had over a given period of time can aid in picking
#stocks that are trending upwards


#NOTE TO US, momenutum is only needed if we change the period for other data as well
#for now both are based on 6 months

def calc_momentum(ticker_returns, period):
    momentum = (ticker_returns.iloc[-1]/ticker_returns.iloc[-period]) - 1
    return momentum





In [10]:
sp500_aligned, tsx_aligned = sp500.align(tsx, join='inner')

# Create benchmark
benchmark_prices = (sp500_aligned + tsx_aligned) / 2
benchmark_returns = benchmark_prices.pct_change().dropna()

risk_free_rate_sortino = 0.04 / 252  # Daily risk-free rate assuming 4% annual
risk_free_rate_alpha = 0.04  # Annual risk-free rate for alpha calculation
market_actual_return = (benchmark_prices.iloc[-1] / benchmark_prices.iloc[0]) - 1 

## Will build loop though stock_data[] and as it goes through each ticker we can calculate the relevant metrics for those tickers
##Apply created functions for metrics to the tickers while we iterate through the loop

alpha_results = []

for ticker in tickers_list:
    stock_close = stock_data[ticker]['Close'].dropna()
    stock_close.index = stock_close.index.tz_localize(None)
    stock_returns = stock_close.pct_change().dropna()

    beta = calc_beta(stock_returns, benchmark_returns)

    
    stock_actual_return = (stock_close.iloc[-1] / stock_close.iloc[0]) - 1
    alpha = calc_alpha(stock_actual_return, risk_free_rate_alpha, market_actual_return, beta)

    sortino = calc_sortino(stock_returns, risk_free_rate_sortino)
    momentum = calc_momentum(stock_close, period=63) #3 months
    
    # Append to results list
    alpha_results.append({
        'Ticker': ticker,
        'Alpha': alpha,
        'Beta': beta,
        'Sortino': sortino,
        'Return': stock_actual_return,
        'Momentum': momentum
    })

    print(f"✓ {ticker}: Alpha={alpha:.4f}, Beta={beta:.2f}, Sortino={sortino:.2f}, Momentum={momentum:.2f}")

# Convert to DataFrame
results_df = pd.DataFrame(alpha_results)

# Sort by Alpha (highest first)
results_df = results_df.sort_values('Alpha', ascending=False)

# Display
print("\n=== Top Stocks by Alpha ===")
results_df.reset_index(drop=True, inplace=True)
results_df
    


    

✓ AAPL: Alpha=0.4684, Beta=1.28, Sortino=0.06, Momentum=0.19
✓ ABBV: Alpha=1.4830, Beta=0.40, Sortino=0.07, Momentum=0.15
✓ ABT: Alpha=-0.2150, Beta=0.62, Sortino=0.01, Momentum=-0.02
✓ ACN: Alpha=-0.7703, Beta=1.12, Sortino=0.00, Momentum=-0.01
✓ HSBC: Alpha=1.8232, Beta=1.04, Sortino=0.08, Momentum=0.06
✓ AIG: Alpha=0.2642, Beta=1.15, Sortino=0.05, Momentum=-0.07
✓ AMZN: Alpha=-0.7708, Beta=1.56, Sortino=0.02, Momentum=-0.03
✓ AXP: Alpha=1.1003, Beta=1.49, Sortino=0.08, Momentum=0.11
✓ BA: Alpha=-1.2730, Beta=1.46, Sortino=-0.00, Momentum=-0.23
✓ BAC: Alpha=0.1033, Beta=1.33, Sortino=0.05, Momentum=0.03
✓ BB.TO: Alpha=-1.5499, Beta=1.76, Sortino=0.02, Momentum=0.14
✓ BIIB: Alpha=-0.9582, Beta=0.86, Sortino=-0.01, Momentum=0.28
✓ BK: Alpha=1.2694, Beta=1.20, Sortino=0.08, Momentum=0.03
✓ SAP: Alpha=0.3316, Beta=1.12, Sortino=0.06, Momentum=-0.12
✓ BLK: Alpha=-0.4730, Beta=1.48, Sortino=0.03, Momentum=-0.11
✓ BMY: Alpha=-0.4168, Beta=0.39, Sortino=-0.01, Momentum=-0.01
✓ C: Alpha=0.130

,Ticker,Alpha,Beta,Sortino,Return,Momentum
0,LLY,6.312200,0.615299,0.120825,6.808205,0.445566
1,SAN,2.280378,1.419973,0.088820,3.372737,0.065308
2,HSBC,1.823150,1.037475,0.083009,2.632035,0.059425
3,ABBV,1.482985,0.402364,0.066320,1.821181,0.146670
4,CAT,1.400808,1.375289,0.083708,2.460052,0.278765
5,BK,1.269369,1.204739,0.078375,2.202215,0.026743
6,PM,1.221095,0.457944,0.069496,1.600483,-0.057521
7,AXP,1.100327,1.491619,0.075224,2.245784,0.107697
8,MO,0.826690,0.374667,0.048372,1.144361,-0.109005
9,RY.TO,0.807880,0.770341,0.085260,1.418790,0.129116


In [11]:
##USING METRICS TO PICK TOP x STOCKS TO INVEST IN

results_df['Alpha_rank'] = results_df['Alpha'].rank(ascending=False)
results_df['Sortino_rank'] = results_df['Sortino'].rank(ascending=False)
results_df['Momentum_rank'] = results_df['Momentum'].rank(ascending=False)

##Beta close to 1.15 is ideal, so we rank based on distance from 1.15
##ascending True means closest to 1.15 gets rank 1, lowest distance to greatest distance
results_df['Beta_rank'] = (results_df['Beta'] - 1.15).abs().rank(ascending=True)

# Score (simple sum - equal weight)
# Different metrics are weighted differently
# Alpha: 30%
# Sortino: 35%
# Momentum: 30%
# Beta: 5%
# This is because sortino is considered more important for risk-adjusted returns
# momentum and alpha are growth indicators 
# Need markdown for this 
results_df['Composite_Score'] = (
    (0.30 * results_df['Alpha_rank']) + 
    (0.35 * results_df['Sortino_rank']) + 
    (0.30 * results_df['Momentum_rank']) + 
    (0.05 * results_df['Beta_rank'])
)

# Sort and select
# Create new dataframe with just ranks and composite score
scored_df = results_df[['Ticker', 'Alpha_rank', 'Sortino_rank', 'Momentum_rank', 'Beta_rank', 'Composite_Score']].copy()

# Sort by composite score
scored_df = scored_df.sort_values('Composite_Score')

scored_df.reset_index(drop=True, inplace=True)

scored_df


,Ticker,Alpha_rank,Sortino_rank,Momentum_rank,Beta_rank,Composite_Score
0,LLY,1.0,1.0,1.0,24.0,2.15
1,CAT,5.0,4.0,4.0,11.0,4.65
2,SAN,2.0,2.0,16.0,15.0,6.85
3,ABBV,4.0,9.0,7.0,31.0,8.00
4,HSBC,3.0,5.0,17.0,6.0,8.05
5,RY.TO,10.0,3.0,10.0,20.0,8.05
6,AAPL,12.0,10.0,5.0,7.0,8.95
7,AXP,8.0,7.0,11.0,19.0,9.10
8,BK,6.0,6.0,21.0,5.0,10.45
9,TD.TO,11.0,12.0,8.0,21.0,10.95


In [12]:
## TOP X STOCKS TO INVEST IN BASED ON METRICS ##
num_stocks_to_invest = 12

top_x_stocks_beforeCHECK = scored_df.head(num_stocks_to_invest)
top_x_stocks_beforeCHECK


,Ticker,Alpha_rank,Sortino_rank,Momentum_rank,Beta_rank,Composite_Score
0,LLY,1.0,1.0,1.0,24.0,2.15
1,CAT,5.0,4.0,4.0,11.0,4.65
2,SAN,2.0,2.0,16.0,15.0,6.85
3,ABBV,4.0,9.0,7.0,31.0,8.00
4,HSBC,3.0,5.0,17.0,6.0,8.05
5,RY.TO,10.0,3.0,10.0,20.0,8.05
6,AAPL,12.0,10.0,5.0,7.0,8.95
7,AXP,8.0,7.0,11.0,19.0,9.10
8,BK,6.0,6.0,21.0,5.0,10.45
9,TD.TO,11.0,12.0,8.0,21.0,10.95


In [13]:
### OVERALL PORTFOLIO CHECK. MAKING SURE PORTFOLIO MEETS 

### BASIC REQUIREMENTS:
### 1. NO MORE THEN 40% IN A SINGLE SECTOR
### 2. ONE SMALL CAP
### 3. ONE LARGE CAP


## first going to sort the valid tickers into new lists of largecap, small cap, and industry
## which i can then use to check if my portfolio works

#argument should be the list of all valid tickers
def sort_smallcap(tickers):

    smallcap_tickers = []

    for ticker in tickers:

        tick_info = yf.Ticker(ticker).fast_info
        market_cap = tick_info.get("marketCap")

        if market_cap is not None and market_cap < 2000000000:
            smallcap_tickers.append(ticker)

    return smallcap_tickers
     


def sort_largecap(tickers):

    largecap_tickers = []

    for ticker in tickers:

        tick_info = yf.Ticker(ticker).fast_info
        market_cap = tick_info.get("marketCap")
        
        if market_cap is not None and market_cap > 10000000000:
            largecap_tickers.append(ticker)

    return largecap_tickers



#portfolio argument should be a list of just the X tickers in the portfolio
################################################# IF WE CHANGE NUMBER OF TICKERS WELL HAVE TO REVISE THIS FUNCTION
#final_tickers argument should be a list of all the valid tickers RANKED from best to worst through all of the analysis above
def check_portfolio(portfolio, final_tickers):
    
    print("\n" + "="*60)
    print("PORTFOLIO VALIDATION CHECK")
    print("="*60)
    print(f"Initial Portfolio: {portfolio}")
    
    smallcap_tickers = sort_smallcap(final_tickers)
    largecap_tickers = sort_largecap(final_tickers)
    
    print(f"\nAvailable Small Cap Stocks (<$2B): {smallcap_tickers}")
    print(f"Available Large Cap Stocks (>$10B): {largecap_tickers}")

    iteration = 0

    small_cap_exists = True
    large_cap_exists = True

    if len(smallcap_tickers) == 0:
        small_cap_exists = False
    
    if len(largecap_tickers) == 0:
        large_cap_exists = False

    swaps = []


    while True:

        if(not small_cap_exists):
            print("\n No Small Cap stocks available in the valid tickers list. \n Final portfolio WILL NOT have a Small Cap stock.")
            
        if(not large_cap_exists):
            print("\n No Large Cap stocks available in the valid tickers list. \n Final portfolio WILL NOT have a Large Cap stock.")


        iteration += 1
        print(f"\n--- Iteration {iteration} ---")
        
        is_smallcap = False
        is_largecap = False
        notfortypercent = True 
        seen_industries = {}
        industry_overforty = None

        for ticker in portfolio:
            if ticker in smallcap_tickers:
                is_smallcap = True
        
            if ticker in largecap_tickers:
                is_largecap = True 

            
            industry = yf.Ticker(ticker).info.get("industry")
            seen_industries[industry] = seen_industries.get(industry, 0) + 1

        # Print industry breakdown
        print("\nIndustry Breakdown:")
        for industry, amount in seen_industries.items():
            percentage = (amount / num_stocks_to_invest) * 100
            print(f"  {industry}: {amount} stocks ({percentage:.0f}%)")
            if amount / num_stocks_to_invest > 0.4:
                notfortypercent = False
                industry_overforty = industry

        # Print validation status
        print(f"\n Has Small Cap (<$2B): {'YES' if is_smallcap else 'NO '}")
        print(f" Has Large Cap (>$10B): {'YES' if is_largecap else 'NO '}")
        print(f" No Sector >40%: {'YES' if notfortypercent else f'NO  ({industry_overforty} is over 40%)'}")

        # ALL conditions are TRUE
        if is_smallcap and is_largecap and notfortypercent:
            print("\n" + "="*60)
            print("PORTFOLIO IS VALID!")
            print("="*60)
            print(f"Final Portfolio: {portfolio}")
            return portfolio

        # ACTUALLY FIXING THE PORTFOLIO     
        if (small_cap_exists and not is_smallcap):
            print("\n Missing Small Cap stock. Finding replacement...")
            for tick in smallcap_tickers:
                if tick not in portfolio:
                    removed = portfolio.pop(-1)
                    portfolio.append(tick)
                    print(f"   ➜ Removed: {removed}")
                    print(f"   ➜ Added: {tick} (Small Cap)")
                    break
            continue
        elif (not small_cap_exists):
            print("NO SMALL CAP. CONTINUING with large cap and industry check.")
            

        if (large_cap_exists and not is_largecap):
            print("\n Missing Large Cap stock. Finding replacement...")
            for tick in largecap_tickers:
                if tick not in portfolio:
                    removed = portfolio.pop(-1)
                    portfolio.append(tick)
                    print(f"   ➜ Removed: {removed}")
                    print(f"   ➜ Added: {tick} (Large Cap)")
                    break

            continue
        elif (not large_cap_exists):
            print("NO LARGE CAP. CONTINUING with industry check.")

        if not notfortypercent and industry_overforty is not None:
            print(f"\n Sector '{industry_overforty}' is over 40%. Finding replacement...")
            
            worstposition = -1
            removeworst = None
            swapwith = None

            for tick in portfolio:
                tick_industry = yf.Ticker(tick).info.get("industry")
                if tick_industry == industry_overforty:
                    position = final_tickers.index(tick)
                    if position > worstposition:
                        worstposition = position
                        removeworst = tick

            for tick in final_tickers:
                if tick not in portfolio and tick != removeworst and tick not in swaps:
                    tick_industry = yf.Ticker(tick).info.get("industry")
                    if tick_industry != industry_overforty:
                        swapwith = tick
                        break

            if removeworst is not None and swapwith is not None:
                swap_industry = yf.Ticker(swapwith).info.get("industry")
                portfolio.remove(removeworst)
                portfolio.append(swapwith)
                print(f"   ➜ Removed: {removeworst} (Sector: {industry_overforty})")
                print(f"   ➜ Added: {swapwith} (Sector: {swap_industry})")

                swaps.append(removeworst)
                swaps.append(swapwith)

            continue

        else:
            return portfolio
        



In [14]:
full_1st = scored_df['Ticker'].tolist()
portfolio_lst = top_x_stocks_beforeCHECK['Ticker'].tolist()

top_x_stocks = check_portfolio(portfolio_lst, full_1st)

##THE LIST HERE should go to the variable top_x_tickers below
#Whatever list is outputted after the check for volume and industry, should move on to the optimization code
#as a list and set equal to top_x_tickers



PORTFOLIO VALIDATION CHECK
Initial Portfolio: ['LLY', 'CAT', 'SAN', 'ABBV', 'HSBC', 'RY.TO', 'AAPL', 'AXP', 'BK', 'TD.TO', 'MRK', 'C']

Available Small Cap Stocks (<$2B): ['FLGT']
Available Large Cap Stocks (>$10B): ['LLY', 'CAT', 'SAN', 'ABBV', 'HSBC', 'RY.TO', 'AAPL', 'AXP', 'BK', 'TD.TO', 'MRK', 'C', 'PM', 'KO', 'BAC', 'SAP', 'AIG', 'MO', 'LMT', 'SHOP.TO', 'UNH', 'BIIB', 'USB', 'ABT', 'UNP', 'QCOM', 'BLK', 'PEP', 'PG', 'AMZN', 'UPS', 'ACN', 'BMY', 'CL', 'PFE', 'TXN', 'T.TO', 'BA', 'PYPL']

--- Iteration 1 ---

Industry Breakdown:
  Drug Manufacturers - General: 3 stocks (25%)
  Farm & Heavy Construction Machinery: 1 stocks (8%)
  Banks - Diversified: 6 stocks (50%)
  Consumer Electronics: 1 stocks (8%)
  Credit Services: 1 stocks (8%)

 Has Small Cap (<$2B): NO 
 Has Large Cap (>$10B): YES
 No Sector >40%: NO  (Banks - Diversified is over 40%)

 Missing Small Cap stock. Finding replacement...
   ➜ Removed: C
   ➜ Added: FLGT (Small Cap)

--- Iteration 2 ---

Industry Breakdown:
  D

In [15]:
# Get the top x tickers

# Get the actual metric values for top x stocks from results_df
top_x_metrics = results_df[results_df['Ticker'].isin(top_x_stocks)].copy()

# Build prices DataFrame for top x stocks
prices_df = pd.DataFrame()
for ticker in top_x_stocks:
    stock_close = stock_data[ticker]['Close'].dropna()
    stock_close.index = stock_close.index.tz_localize(None)
    prices_df[ticker] = stock_close

# Align all columns to same dates
prices_df = prices_df.dropna()
prices_df


,LLY,CAT,SAN,ABBV,HSBC,RY.TO,AAPL,AXP,BK,MRK,FLGT,PM
Date,,,,,,,,,,,,
2020-11-23,136.064560,158.928375,2.331492,84.081810,18.970081,87.514839,110.827530,109.135429,33.370651,65.401237,43.599998,59.846577
2020-11-24,137.507019,160.856094,2.484099,85.546532,19.747850,89.263481,112.112511,113.177826,35.223129,65.287170,36.939999,60.932129
2020-11-25,137.364670,158.792007,2.467142,85.744019,19.814512,89.263481,112.949692,113.356445,34.946129,65.222008,36.509998,60.049633
2020-11-27,139.917450,159.201187,2.467142,86.311798,19.918215,89.636414,113.494812,113.365845,34.582561,65.059082,40.630001,59.276459
2020-11-30,138.218765,157.846298,2.390839,86.056732,19.177483,87.896080,115.889511,111.485641,33.864063,65.490852,44.919998,59.159313
...,...,...,...,...,...,...,...,...,...,...,...,...
2025-11-17,1021.700012,552.049988,10.400000,234.000000,71.220001,206.929993,267.459991,341.250000,106.769997,92.860001,28.809999,152.759995
2025-11-18,1030.050049,546.880005,10.170000,233.869995,69.589996,208.380005,267.440002,340.660004,107.500000,96.430000,29.100000,155.850006
2025-11-19,1049.599976,553.109985,10.280000,232.910004,69.139999,208.509995,268.559998,344.640015,107.510002,95.050003,29.000000,155.610001


In [16]:
print(f"\n{'='*60}")
print("BLACK-LITTERMAN PORTFOLIO OPTIMIZATION")
print(f"{'='*60}")

# Calculate market baseline
years = 5
market_total_return = (benchmark_prices.iloc[-1] / benchmark_prices.iloc[0]) - 1
market_baseline = (1 + market_total_return) ** (1/years) - 1

print(f"Market Baseline Annual Return: {market_baseline:.2%}") 

# Create views and confidences
viewdict = {}
confidences = []

for _, row in top_x_metrics.iterrows():
    ticker = row['Ticker']
    alpha_annual = row['Alpha']
    momentum_3mo = row['Momentum']
    momentum_annual = (1 + momentum_3mo) ** 4 - 1
    
    expected_return = market_baseline + alpha_annual + (0.5 * momentum_annual)
    expected_return = np.clip(expected_return, 0.05, 0.30)
    viewdict[ticker] = expected_return
    
    alpha_norm = row['Alpha'] / top_x_metrics['Alpha'].max()
    sortino_norm = row['Sortino'] / top_x_metrics['Sortino'].max()
    combined = (alpha_norm + sortino_norm) / 2
    confidence = 0.5 + combined * 0.5
    confidences.append(confidence)

# Black-Litterman with prior
S = risk_models.sample_cov(prices_df)

bl = BlackLittermanModel(
    S, 
    absolute_views=viewdict, 
    view_confidences=confidences,
    pi='equal'  # FIX 1: Add prior
)

ret_bl = bl.bl_returns()
S_bl = bl.bl_cov()

# Optimize
risk_free = 0.04
lower_bound_weight = (100 / (2 * num_stocks_to_invest)) / 100 # 5% for 10 stocks

ef = EfficientFrontier(ret_bl, S_bl, weight_bounds=(lower_bound_weight, 0.15))
weights = ef.max_sharpe(risk_free_rate=risk_free)
cleaned_weights = ef.clean_weights()

# Get performance with same risk-free rate
performance = ef.portfolio_performance(
    verbose=False, 
    risk_free_rate=risk_free  # FIX 2: Use same rate
)

expected_return, expected_volatility, sharpe_ratio = performance

# Create DataFrame
weights_df = pd.DataFrame({
    'Ticker': list(cleaned_weights.keys()),
    'Weight': list(cleaned_weights.values())
}).sort_values('Weight', ascending=False)

weights_df['Weight'] = weights_df['Weight'] / weights_df['Weight'].sum()

print(f"\n{'='*60}")
print("OPTIMIZED PORTFOLIO WEIGHTS")
print(f"{'='*60}")
print(weights_df.to_string(index=False))

weights_sum = weights_df['Weight'].sum()

print(f"Sum: {weights_sum}")

print(f"\nExpected Annual Return:    {expected_return:.2%}")
print(f"Expected Annual Volatility: {expected_volatility:.2%}")
print(f"Expected Sharpe Ratio:      {sharpe_ratio:.2f}")



BLACK-LITTERMAN PORTFOLIO OPTIMIZATION
Market Baseline Annual Return: 12.24%

OPTIMIZED PORTFOLIO WEIGHTS
Ticker   Weight
 RY.TO 0.150002
   MRK 0.150002
    PM 0.150002
  ABBV 0.141221
  HSBC 0.054581
   LLY 0.054191
   CAT 0.050001
   SAN 0.050001
  AAPL 0.050001
   AXP 0.050001
    BK 0.050001
  FLGT 0.050001
Sum: 0.9999999999999996

Expected Annual Return:    24.63%
Expected Annual Volatility: 14.03%
Expected Sharpe Ratio:      1.47


In [17]:
''''

### OVERALL PORTFOLIO CHECK. MAKING SURE PORTFOLIO MEETS 

### BASIC REQUIREMENTS:
### 1. NO MORE THEN 40% IN A SINGLE SECTOR
### 2. ONE SMALL CAP
### 3. ONE LARGE CAP


## first going to sort the valid tickers into new lists of largecap, small cap, and industry
## which i can then use to check if my portfolio works

#argument should be the list of all valid tickers
def sort_smallcap(tickers):

    smallcap_tickers = []

    for ticker in tickers:

        tick_info = yf.Ticker(ticker).fast_info
        market_cap = tick_info.get("marketCap")

        if market_cap is not None and market_cap < 2000000000:
            smallcap_tickers.append(ticker)

    return smallcap_tickers
     


def sort_largecap(tickers):

    largecap_tickers = []

    for ticker in tickers:

        tick_info = yf.Ticker(ticker).fast_info
        market_cap = tick_info.get("marketCap")
        
        if market_cap is not None and market_cap > 10000000000:
            largecap_tickers.append(ticker)

    return largecap_tickers



#portfolio argument should be a list of just the TEN tickers in the portfolio
################################################# IF WE CHANGE NUMBER OF TICKERS WELL HAVE TO REVISE THIS FUNCTION
#final_tickers argument should be a list of all the valid tickers RANKED from best to worst through all of the analysis above
def check_portfolio(portfolio, final_tickers):
    

    smallcap_tickers = sort_smallcap(final_tickers)
    largecap_tickers = sort_largecap(final_tickers)

    while True:

        is_smallcap = False
        is_largecap = False
        notfortypercent = True 
        seen_industries = {}
        industry_overforty = None



        for ticker in portfolio:

            if ticker in smallcap_tickers:
                is_smallcap = True
        
            if ticker in largecap_tickers:
                is_largecap = True 

            #keeping track of how many times weve seen a certain industry
            industry = yf.Ticker(ticker).info.get("industry")
            seen_industries[industry] += 1

        for industry, amount in seen_industries.items():
            if amount / num_stocks_to_invest > 0.4:
                notfortypercent = False
                industry_overforty = industry

        if is_smallcap and is_largecap and industry_overforty:
            print("Portfolio is Valid.")
            return portfolio
        


        #ACTUALLY FIXING THE PORTFOLIO

        #the logic for small cap and large cap is this:
        #if our booleans are true then there is NO small/large cap in our portfolio so
        #we remove the last stock in our actual portfolio, because 1-9 are better then the tenth stock based of our algorithm
        #then we take the first ticker in our sorted list of small/large cap tickers that is NOT in our portfolio, 
        # (technically there shouldnt be a small/large cap in there but this is a good extra safet check)
        #and we put it in to the portfolio back at the 10th position

        if not is_smallcap:
            for tick in smallcap_tickers:
                if tick not in portfolio:
                    portfolio.pop(-1)
                    portfolio.append(tick)
                    break

            continue

        if not is_largecap:
            for tick in largecap_tickers:
                if tick not in portfolio:
                    portfolio.pop(-1)
                    portfolio.append(tick)
                    break

            continue


        #the logic for the industry check is this:
        #if there is an industry worth over forty percent AND we have the industry name, which we should then
        #we take out the last occurene of that industry in our portfolio, again because its ranked high to low, the latter stocks are non-optimal picks
        #so we remove the last occurece, and then we look in the tickers for the first one that is NOT in the portfolio and NOT in the same industry
        #and then we replace it by appending it to the end of the portfolio
        
    
        if not notfortypercent and industry_overforty is not None:

            worstposition = -1 #last index
            removeworst = None
            swapwith = None

            for tick in portfolio:

                tick_industry = yf.Ticker(tick).info.get("industry")

                if tick_industry == industry_overforty:
                    position = final_tickers.index(tick)

                    if position > worstposition:
                        worstposition = position
                        removeworst = tick

        
            for tick in final_tickers:
                if tick not in portfolio:
                    tick_industry = yf.Ticker(tick).info.get("industry")

                    if tick_industry != industry_overforty:
                        swapwith = tick
                        break


            if removeworst != None and swapwith != None:
                portfolio.remove(removeworst)
                portfolio.append(swapwith)

            continue


        else:
            return portfolio
        
'''




'\'\n\n### OVERALL PORTFOLIO CHECK. MAKING SURE PORTFOLIO MEETS \n\n### BASIC REQUIREMENTS:\n### 1. NO MORE THEN 40% IN A SINGLE SECTOR\n### 2. ONE SMALL CAP\n### 3. ONE LARGE CAP\n\n\n## first going to sort the valid tickers into new lists of largecap, small cap, and industry\n## which i can then use to check if my portfolio works\n\n#argument should be the list of all valid tickers\ndef sort_smallcap(tickers):\n\n    smallcap_tickers = []\n\n    for ticker in tickers:\n\n        tick_info = yf.Ticker(ticker).fast_info\n        market_cap = tick_info.get("marketCap")\n\n        if market_cap is not None and market_cap < 2000000000:\n            smallcap_tickers.append(ticker)\n\n    return smallcap_tickers\n     \n\n\ndef sort_largecap(tickers):\n\n    largecap_tickers = []\n\n    for ticker in tickers:\n\n        tick_info = yf.Ticker(ticker).fast_info\n        market_cap = tick_info.get("marketCap")\n        \n        if market_cap is not None and market_cap > 10000000000:\n        

In [18]:
### FINAL CODE PORTFOLIO GENERATION



money_cad = 1000000
maxfee_usd = 2.15 
numstocks = num_stocks_to_invest


##takes dataframe with columns "Ticker" and "Weight" as a DECIMAL
def produce_finaldf(df_tickers_and_weight):

    #get exchange rates for both directions
    usd_to_cad = yf.Ticker("CAD=X").fast_info["lastPrice"]
    cad_to_usd = 1 / usd_to_cad

    max_fees_cad = maxfee_usd * usd_to_cad
    investable_cad = money_cad - max_fees_cad

    final_tickers = []
    final_price = []
    final_currency = []
    final_shares = []
    final_value_cad = []
    final_weights = []

    for indx, row in df_tickers_and_weight.iterrows():
        ticker = row["Ticker"]
        weight_decimal = row["Weight"]

        spend_amnt_cad = investable_cad * weight_decimal

        info = yf.Ticker(ticker).fast_info
        price = info["lastPrice"]
        currency = info.get("currency", "USD")

        ##NEED TO CHECK IF USD OR CAD ARE THE ONLY POSSIBLE CURRENCIES

        #if the stock is in usd, convert its price into cad so its simpler to buy it since our money is in cad
        if currency == "USD":
            price_cad = price * usd_to_cad
        else:
            price_cad = price

        shares = spend_amnt_cad / price_cad
        value_cad = shares * price_cad

        final_tickers.append(ticker)
        final_price.append(price)
        final_currency.append(currency)
        final_shares.append(shares)
        final_value_cad.append(value_cad)
        final_weights.append(round(weight_decimal * 100,2)) #to make a percent
        
    
    total_shares = sum(final_shares)
    fee_usd_shares = total_shares * 0.001
    fee_usd = min(maxfee_usd, fee_usd_shares)
    fee_cad = fee_usd * usd_to_cad

    df_final = pd.DataFrame({
        "Ticker": final_tickers,
        "Price": final_price,
        "Currency": final_currency,
        "Shares": final_shares,
        "Value (CAD)": final_value_cad,
        "Weight (%)": final_weights})
    
    total_invested = sum(final_value_cad)
    final_spent = total_invested + fee_cad
    leftover_cash = money_cad - final_spent

    
    print("\n\nTotal Portfolio Value (CAD): $", round(total_invested,2))
    print("Fee Charged (USD): $", fee_usd, "| Fee Charged (CAD): $", round(fee_cad,2))
    print("Total Spent Including Fees (CAD): $", round(final_spent,2))
    ##print("Remaining Cash (CAD): $", leftover_cash)

    return df_final



print(weights_df)
produce_finaldf(weights_df)




   Ticker    Weight
5   RY.TO  0.150002
9     MRK  0.150002
11     PM  0.150002
3    ABBV  0.141221
4    HSBC  0.054581
0     LLY  0.054191
1     CAT  0.050001
2     SAN  0.050001
6    AAPL  0.050001
7     AXP  0.050001
8      BK  0.050001
10   FLGT  0.050001


Total Portfolio Value (CAD): $ 999996.97
Fee Charged (USD): $ 2.15 | Fee Charged (CAD): $ 3.03
Total Spent Including Fees (CAD): $ 1000000.0


,Ticker,Price,Currency,Shares,Value (CAD),Weight (%)
0,RY.TO,211.639999,CAD,708.755651,150001.045511,15.00
1,MRK,98.330002,USD,1082.442402,150001.045511,15.00
2,PM,155.610001,USD,683.995649,150001.045511,15.00
3,ABBV,237.229996,USD,422.402219,141220.984314,14.12
4,HSBC,68.896103,USD,562.132195,54580.380427,5.46
5,LLY,1063.025024,USD,36.172227,54190.377708,5.42
6,CAT,550.020020,USD,64.504660,50000.348504,5.00
7,SAN,10.195000,USD,3480.025063,50000.348504,5.00
8,AAPL,272.130005,USD,130.374651,50000.348504,5.00
9,AXP,354.216003,USD,100.161636,50000.348504,5.00


## Contribution Declaration

The following team members made a meaningful contribution to this assignment:

Insert Names Here.